# Satellite-Derived Bathymetry Model Deployment

This notebook demonstrates how to:
1. Load trained models
2. Apply models to new Sentinel-2 imagery
3. Visualize and export bathymetry predictions
4. Analyze prediction uncertainties

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import folium
from branca.colormap import LinearColormap

# Add project root to path
project_dir = Path().absolute().parent
if project_dir.name != 'sdb_project':
    project_dir = project_dir / 'sdb_project'
sys.path.append(str(project_dir))

# Load configuration
config_path = project_dir / 'config' / 'location_config.json'
with open(config_path) as f:
    config = json.load(f)

# Set up region-specific paths
region_slug = config['region_name'].lower().replace(' ', '_')
data_dir = project_dir / 'data' / 'processed' / region_slug
models_dir = project_dir / 'models'
output_dir = project_dir / 'outputs' / region_slug / 'deployment'

# Create output directories
for dir_path in [output_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Import our modules
from src.sdb_model import SDBModel
from src.visualize import plot_bathymetry_2d, create_3d_surface

print("Deploying models for region:", config['region_name'])
print("Area of Interest:")
print(f"Latitude:  {config['aoi']['min_lat']}° to {config['aoi']['max_lat']}°")
print(f"Longitude: {config['aoi']['min_lon']}° to {config['aoi']['max_lon']}°")
print("\nDirectories:")
print(f"Data directory: {data_dir}")
print(f"Models directory: {models_dir}")
print(f"Output directory: {output_dir}")

## Load Configuration and Data

In [ ]:
# Load region configuration
config_path = project_dir / 'config' / 'location_config.json'
with open(config_path) as f:
    config = json.load(f)

region_slug = config['region_name'].lower().replace(' ', '_')

# Setup paths
data_dir = project_dir / 'data' / 'processed' / region_slug
output_dir = project_dir / 'outputs' / region_slug
model_dir = project_dir / 'models'

# Load preprocessed data
features = np.load(data_dir / 'features.npy')
water_mask = np.load(data_dir / 'water_mask.npy')

print(f"Processing region: {config['region_name']}")
print(f"Features shape: {features.shape}")

## Load Trained Models

In [ ]:
# Initialize SDB model handler
sdb_model = SDBModel(output_dir)

# Load feature scaler
scaler = sdb_model.load_scaler()

# Load all models
models = {
    'linear_regression': sdb_model.load_linear_regression(),
    'random_forest': sdb_model.load_random_forest(),
    'xgboost': sdb_model.load_xgboost()
}

print("Loaded models:", list(models.keys()))

## Make Predictions

Apply models to the entire study area

In [ ]:
# Prepare data for prediction
valid_pixels = water_mask.reshape(-1)
X = features.reshape(-1, features.shape[-1])[valid_pixels]
X_scaled = scaler.transform(X)

# Make predictions
predictions = {}
for name, model in models.items():
    # Initialize prediction array
    pred = np.full(water_mask.shape, np.nan)
    
    # Make predictions for water pixels
    pred_water = model.predict(X_scaled)
    pred.reshape(-1)[valid_pixels] = pred_water
    
    predictions[name] = pred
    print(f"{name}: Depth range {pred_water.min():.1f}m to {pred_water.max():.1f}m")

## Create Bathymetry Maps

In [ ]:
# Plot individual model predictions
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
vmin, vmax = -30, 0  # Depth range for visualization

for idx, (name, pred) in enumerate(predictions.items()):
    ax = axes[idx]
    im = ax.imshow(pred, cmap='viridis', vmin=vmin, vmax=vmax)
    ax.set_title(name.replace('_', ' ').title())
    plt.colorbar(im, ax=ax, label='Depth (m)')
    ax.axis('off')

plt.tight_layout()
plt.savefig(output_dir / 'model_predictions.png')
plt.show()

## Create Ensemble Prediction

In [ ]:
# Calculate ensemble prediction (weighted average)
# Load model metrics to use as weights
metrics_path = output_dir / 'model_metrics.json'
with open(metrics_path) as f:
    metrics = json.load(f)

# Use R² scores as weights
weights = {name: metrics[name]['r2_score'] for name in predictions.keys()}
total_weight = sum(weights.values())
weights = {k: v/total_weight for k, v in weights.items()}

# Calculate weighted average
ensemble_pred = np.zeros_like(list(predictions.values())[0])
for name, pred in predictions.items():
    ensemble_pred += pred * weights[name]

# Save ensemble prediction
np.save(output_dir / 'ensemble_prediction.npy', ensemble_pred)

print("Ensemble weights:")
for name, weight in weights.items():
    print(f"{name}: {weight:.3f}")

## Create Interactive Map

In [ ]:
# Create interactive map
center_lat = (config['aoi']['min_lat'] + config['aoi']['max_lat']) / 2
center_lon = (config['aoi']['min_lon'] + config['aoi']['max_lon']) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add bathymetry layer
valid_mask = ~np.isnan(ensemble_pred)
depths = ensemble_pred[valid_mask]
lats = np.linspace(config['aoi']['min_lat'], config['aoi']['max_lat'], ensemble_pred.shape[0])
lons = np.linspace(config['aoi']['min_lon'], config['aoi']['max_lon'], ensemble_pred.shape[1])
lat_grid, lon_grid = np.meshgrid(lats, lons)

# Create colormap
colormap = LinearColormap(
    colors=['darkblue', 'blue', 'lightblue'],
    vmin=depths.min(),
    vmax=0
)

# Add points to map
for i in range(0, len(depths), 100):  # Sample every 100th point for performance
    folium.CircleMarker(
        location=[lat_grid.flatten()[valid_mask.flatten()][i],
                 lon_grid.flatten()[valid_mask.flatten()][i]],
        radius=2,
        color=colormap(depths[i]),
        fill=True
    ).add_to(m)

colormap.add_to(m)
colormap.caption = 'Depth (m)'

# Save map
m.save(output_dir / 'bathymetry_map.html')

# Display map in notebook
m

## Export Results

Save predictions and uncertainty estimates

In [ ]:
# Calculate prediction uncertainty (standard deviation between models)
model_preds = np.stack([pred for pred in predictions.values()])
uncertainty = np.nanstd(model_preds, axis=0)

# Save results
results = {
    'region_name': config['region_name'],
    'aoi': config['aoi'],
    'model_weights': weights,
    'depth_statistics': {
        'min_depth': float(np.nanmin(ensemble_pred)),
        'max_depth': float(np.nanmax(ensemble_pred)),
        'mean_depth': float(np.nanmean(ensemble_pred)),
        'median_depth': float(np.nanmedian(ensemble_pred))
    },
    'uncertainty_statistics': {
        'mean_uncertainty': float(np.nanmean(uncertainty)),
        'max_uncertainty': float(np.nanmax(uncertainty))
    }
}

# Save results to JSON
with open(output_dir / 'bathymetry_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to:", output_dir)
print("\nDepth Statistics:")
for key, value in results['depth_statistics'].items():
    print(f"{key}: {value:.2f}m")

print("\nUncertainty Statistics:")
for key, value in results['uncertainty_statistics'].items():
    print(f"{key}: {value:.2f}m")